In [1]:
%matplotlib inline
from matplotlib import style
style.use('fivethirtyeight')
import matplotlib.pyplot as plt

In [2]:
import numpy as np
import pandas as pd
import datetime as dt

# Reflect Tables into SQLAlchemy ORM

In [3]:
# Python SQL toolkit and Object Relational Mapper
import sqlalchemy
from sqlalchemy.ext.automap import automap_base
from sqlalchemy.orm import Session
from sqlalchemy import create_engine, func

In [4]:
# create engine to hawaii.sqlite
engine = create_engine("sqlite:///hawaii.sqlite")

In [5]:
# reflect an existing database into a new model
Base = automap_base()

# reflect the tables

Base.prepare(engine, reflect=True)


In [6]:
# View all of the classes that automap found
Base.classes.keys()

[]

In [7]:
# Save references to each table
measurement = Base.classes.measurement
station = Base.classes.station


AttributeError: measurement

In [8]:
# Create our session (link) from Python to the DB

session = Session(engine)
inspector = inspect(engine)

# --- check columns from the two tables ---
m_columns = inspector.get_columns('measurement')
for column in m_columns:
    print(column["name"], column["type"])
print("-"*12)
s_columns = inspector.get_columns('station')
for c in s_columns:
     print(c["name"], c["type"])


NameError: name 'inspect' is not defined

# Exploratory Precipitation Analysis

In [ ]:
# Find the most recent date in the data set.

In [9]:
# Design a query to retrieve the last 12 months of precipitation data and plot the results. 
# Starting from the most recent data point in the database. 

# Calculate the date one year from the last date in data set.
last_date = session.query(measurement.date).order_by(measurement.date.desc()).first()[0]

# Perform a query to retrieve the data and precipitation scores
prcp_data = session.query(measurement.date, measurement.prcp).\
            filter((measurement.date >= date_oneyearago)\
                    & (measurement.date <= last_dt)).all()

# Save the query results as a Pandas DataFrame and set the index to the date column
prcp_df = pd.DataFrame(prcp_data).sort_values("date").set_index("date")

# Sort the dataframe by date
last_date = session.query(measurement.date).order_by(measurement.date.desc()).first()[0]

# Use Pandas Plotting with Matplotlib to plot the data
prcp_df.plot(figsize=(10,6), fontsize=12, rot=90)
plt.title(f"Precipitation (inches) from {date_oneyearago} to {last_dt}", fontsize=18)
plt.xlabel("Date")
plt.ylabel("Precipitation (inches)")

plt.tight_layout()
plt.savefig("Images/prcp.png")
plt.show()



NameError: name 'measurement' is not defined

In [10]:
# Use Pandas to calcualte the summary statistics for the precipitation data
prcp_df.describe()


NameError: name 'prcp_df' is not defined

# Exploratory Station Analysis

In [11]:
# Design a query to calculate the total number stations in the dataset
num_station = session.query(station).count()

print("Counting from station table:")
print(f"There are {num_station} stations in the dataset.\n")

# --- using sqlalchemy distinct ---
from sqlalchemy import distinct
distinct_stations = session.query(distinct(measurement.station)).count()

print("Counting from measurement table (using distinct):")
print(f"There are {distinct_stations} stations in the dataset.")

NameError: name 'station' is not defined

In [12]:
# Design a query to find the most active stations (i.e. what stations have the most rows?)
# List the stations and the counts in descending order.
active_stations = session.query(measurement.station, func.count(measurement.station)).\
                  order_by(func.count(measurement.station).desc()).\
                  group_by(measurement.station).\
                  all()

# --- display the active stations ---
active_stations

NameError: name 'measurement' is not defined

In [13]:
# Using the most active station id from the previous query, calculate the lowest, highest, and average temperature.
# --- identify most active station ---
most_active_station = active_stations[0][0]
print (f"The most active station is {most_active_station}.")
print("---------------------------------------------")

# --- calculate lowest temp recorded at the most active station USC00519281 ---
lowest_temp = session.query(func.min(measurement.tobs)).\
              filter(measurement.station == most_active_station).scalar()
              
print(f"Lowest temperature: {lowest_temp} degrees Fahrenheit.")

# --- calculate highest temp recorded at the most active station USC00519281 ---
highest_temp = session.query(func.max(measurement.tobs)).\
              filter(measurement.station == most_active_station).scalar()

print(f"Highest temperature: {highest_temp} degrees Fahrenheit.")

# --- calculate average temp recorded at the most active station USC00519281 ---
avg_temp = session.query(func.avg(measurement.tobs)).\
              filter(measurement.station == most_active_station).scalar()

print(f"Average temperature: {round(avg_temp, 2)} degrees Fahrenheit.")


NameError: name 'active_stations' is not defined

In [14]:
# Using the most active station id
# Query the last 12 months of temperature observation data for this station and plot the results as a histogram
most_active_tobs = pd.DataFrame(session.query(measurement.tobs).\
                                filter((measurement.station == most_active_station)\
                                        & (measurement.date >= date_oneyearago)\
                                        & (measurement.date <= last_dt)).all())

most_active_tobs.plot(kind="hist", figsize=(9,6), bins=12, legend=True)
plt.xticks(fontsize=12)
plt.yticks(fontsize=12)
plt.xlabel("Temperature", fontsize=14)
plt.ylabel("Frequency", fontsize=14)
plt.title(f"Temperature observations (tobs) at {most_active_station}", fontsize=16)

plt.tight_layout()
plt.savefig("Images/tobs.png")
plt.show()


NameError: name 'measurement' is not defined

# Close session

In [15]:
# Close Session
session.close()